<a href="https://colab.research.google.com/github/derekphilipau/machinelearningforartists/blob/main/stylegan2_ada_pytorch_yue_zhou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training "Clay" with StyleGAN

In this tutorial, we will use Yue Zhou's dataset of clay to train a StyleGAN model.

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [ ]:
!nvidia-smi -L

## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install Stylegan2-ada-pytorch Prerequisites

In [3]:
import torch

Verify the next command results in "1".  If not, go back to the beginning and verify you have a GPU runtime.

In [ ]:
torch.cuda.device_count()

In [5]:
import torchvision

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

## Get the StyleGAN code

In [ ]:
!git clone https://github.com/derekphilipau/stylegan2-ada-pytorch.git

In [8]:
mkdir /content/stylegan2-ada-pytorch/datasets

## Get Yue's dataset of images

In [ ]:
cd /content/stylegan2-ada-pytorch/datasets

In [ ]:
!wget https://github.com/derekphilipau/machinelearningforartists/raw/main/yue_zhou_dataset.zip

In [ ]:
!unzip yue_zhou_dataset.zip

In [12]:
!rm -rf __MACOSX

In [13]:
!rm yue_zhou_dataset.zip

In [ ]:
cd /content/stylegan2-ada-pytorch/

## Prepare Yue's dataset for use by StyleGAN

In [ ]:
!python dataset_tool.py --source=./datasets/yue_zhou_dataset --dest=./datasets/yue_zhou_dataset.zip

## Create Folders on Your Google Drive

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will create a new folder on your Google Drive, **MachineLearningForArtists**.  Yue Zhou's training data will be stored in **MachineLearningForArtists/YueZhou**

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists/YueZhou

If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Train from Scratch

This cell block will train StyleGAN from scratch.  Training from scratch is much slower than using *transfer learning* on a previously trained model.  However, the purpose of this tutorial is to a) try training for the first time and b) notice how the progress images develop over time.

In [ ]:
!python train.py --outdir=/content/drive/MyDrive/MachineLearningForArtists/YueZhou --data=./datasets/yue_zhou_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none


## Wait an Hour or Two

Normal training of a dataset can take days.  For this demonstration, we only want to train for a few hours in order to get familiar with the training process and see the initial results.

Results are stored in your Google Drive under drive/MyDrive/MachineLearningForArtists/YueZhou

Each training run is stored in a separate directory.  The initial training run is stored in `00000-yue_zhou...`.  If you run training twice, the second run will be stored in `00001-yue_zhou...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  You should see various "clay" images in this image grid. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.

Since we only want to test a training, once you get to around the 500th iteration, you can stop the above training cell.  Then take a look at the `fakes` images and familiarize yourself with how the training progresses.



## Transfer Learning

Now, let's train the same dataset but use *transfer learning*.  We will use a StyleGAN2 model already pre-trained on faces to train a new model that is trained against Yue Zhou's dataset.

### Get the faces model

In [ ]:
%cd /content/stylegan2-ada-pytorch

In [ ]:
!apt-get install megatools

In [ ]:
!megadl https://mega.nz/#!eQdHkShY!8wyNKs343L7YUjwXlEg3cWjqK2g2EAIdYz5xbkPy3ng

### Prepare old Faces model for StyleGAN2-ada-pytorch


In [ ]:
!python legacy.py \
    --source=/content/stylegan2-ada-pytorch/ffhq-512-avg-tpurun1.pkl \
    --dest=stylegan2-ada-pytorch-ffhq-512.pkl

### Train using the faces model with Yue Zhou's dataset 

In [ ]:
!python train.py --resume=/content/stylegan2-ada-pytorch/stylegan2-ada-pytorch-ffhq-512.pkl --outdir=/content/drive/MyDrive/MachineLearningForArtists/YueZhouFaces --data=./datasets/yue_zhou_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none


## Resume Training from Last Saved Snapshot

If you had to stop training, the browser window closed, or the Colab session timed out (12-24 hours), you can resume training by using a .pkl file stored on your Google Drive.  For Yue Zhou's project, this file will be in `drive -> MyDrive -> MachineLearningForArtists -> YueZhou (or YueZhouFaces) -> 0000X-yue_zhou...`

In [19]:
#@title PKL File Path 
#@markdown Copy & Paste .pkl file path here:
pkl_file = '/content/drive/MyDrive/MachineLearningForArtists/YueZhou/00000-yue_zhou_dataset-mirror-paper512/network-snapshot-000000.pkl' #@param {type:"string"}


In [ ]:
!python train.py --resume=$pkl_file --outdir=/content/drive/MyDrive/MachineLearningForArtists/YueZhouFaces --data=./datasets/yue_zhou_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none
